# Generative Adversarial Networks: SequenceGAN on YelpZIP

In this notebook, I will be recreating the precursor to the current SOTA text-based GAN, SeqGAN, which is the framework that FakeGAN is based off of. You can imagine the flow like:

sequence_gan -> SeqGAN -> FakeGAN -> DeceptGAN (us!)

In this notebook I will be demonstrating how the first iteration, sequence_gan, works, by manually constructing it's own generator and discriminator, even down to the GRU cells. This notebook is based off the authors own demo, where he trains it on Moby Dick. We will instead train it on our own data, namely YelpZIP, in the hopes that it can reach some viable accuracy and gain a deeper understanding of GANs in the process. Let's begin.



In [0]:
from __future__ import print_function
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
from tensorflow.python.ops import tensor_array_ops, control_flow_ops
import numpy as numpy
from google.colab import auth

In [0]:
# assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
# TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
# print('TPU address is', TPU_ADDRESS)

# from google.colab import auth
# auth.authenticate_user()
# with tf.Session(TPU_ADDRESS) as session:
#   print('TPU devices:')
#   pprint.pprint(session.list_devices())

#   # Upload credentials to TPU.
#   with open('/content/adc.json', 'r') as f:
#     auth_info = json.load(f)
#   tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
#   # Now credentials are set for all future sessions on this TPU.


OUTPUT_DIR = 'textGan-v1'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = True #@param {type:"boolean"}
BUCKET = 'lucas0' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://lucas0/textGan-v1 *****


What's in the next cell seems extremely verbose, but on inspection it is simply a manual reconstruction of the Keras layers we are familiar with, down to the gates and activation functions. I imagine the creator of sequence_gan probably did this to gain full control over every single variable in the network. 

The RNN class described below encapsulates both the generator and discriminator variants. The details are convoluted and I don't quite understand every operation, however I have an understanding of how each input to the RNN initializer affect the network structure. These variables are declared in the same cell as main() below, and I have tweaked them according to our data and particular problem.

In [0]:
import tensorflow as tf


def _cumsum(x, length):
    lower_triangular_ones = tf.constant(
        np.tril(np.ones((length, length))),
        dtype=tf.float32)
    return tf.reshape(
            tf.matmul(lower_triangular_ones,
                      tf.reshape(x, [length, 1])),
            [length])


def _backwards_cumsum(x, length):
    upper_triangular_ones = tf.constant(
        np.triu(np.ones((length, length))),
        dtype=tf.float32)
    return tf.reshape(
            tf.matmul(upper_triangular_ones,
                      tf.reshape(x, [length, 1])),
            [length])


class RNN(object):

    def __init__(self, num_emb, emb_dim, hidden_dim,
                 sequence_length, start_token,
                 learning_rate=0.01, reward_gamma=0.9):
        self.num_emb = num_emb
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.sequence_length = sequence_length
        self.start_token = tf.constant(start_token, dtype=tf.int32)
        self.learning_rate = tf.Variable(float(learning_rate), trainable=False)
        self.reward_gamma = reward_gamma
        self.g_params = []
        self.d_params = []

        self.expected_reward = tf.Variable(tf.zeros([self.sequence_length]))

        with tf.variable_scope('generator'):
            self.g_embeddings = tf.Variable(self.init_matrix([self.num_emb, self.emb_dim]))
            self.g_params.append(self.g_embeddings)
            self.g_recurrent_unit = self.create_recurrent_unit(self.g_params)  # maps h_tm1 to h_t for generator
            self.g_output_unit = self.create_output_unit(self.g_params, self.g_embeddings)  # maps h_t to o_t (output token logits)

        with tf.variable_scope('discriminator'):
            self.d_embeddings = tf.Variable(self.init_matrix([self.num_emb, self.emb_dim]))
            self.d_params.append(self.d_embeddings)
            self.d_recurrent_unit = self.create_recurrent_unit(self.d_params)  # maps h_tm1 to h_t for discriminator
            self.d_classifier_unit = self.create_classifier_unit(self.d_params)  # maps h_t to class prediction logits
            self.d_h0 = tf.Variable(self.init_vector([self.hidden_dim]))
            self.d_params.append(self.d_h0)

        self.h0 = tf.placeholder(tf.float32, shape=[self.hidden_dim])  # initial random vector for generator
        self.x = tf.placeholder(tf.int32, shape=[self.sequence_length])  # sequence of indices of true data, not including start token
        self.samples = tf.placeholder(tf.float32, shape=[self.sequence_length])  # random samples from [0, 1]

        # generator on initial randomness
        gen_o = tensor_array_ops.TensorArray(dtype=tf.float32, size=self.sequence_length,
                                             dynamic_size=False, infer_shape=True)
        gen_x = tensor_array_ops.TensorArray(dtype=tf.int32, size=self.sequence_length,
                                             dynamic_size=False, infer_shape=True)
        samples = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length) # placeholder array to store our generated sequences
        samples = samples.unstack(self.samples)
        def _g_recurrence(i, x_t, h_tm1, gen_o, gen_x):
            h_t = self.g_recurrent_unit(x_t, h_tm1)
            o_t = self.g_output_unit(h_t) # output layer before sampling
            sample = samples.read(i)
            o_cumsum = _cumsum(o_t, self.num_emb)  # prepare for sampling
            next_token = tf.to_int32(tf.reduce_min(tf.where(sample < o_cumsum)))  # sample
            x_tp1 = tf.gather(self.g_embeddings, next_token)
            gen_o = gen_o.write(i, tf.gather(o_t, next_token))  # we only need the sampled token's probability
            gen_x = gen_x.write(i, next_token)  # indices, not embeddings
            return i + 1, x_tp1, h_t, gen_o, gen_x

        _, _, _, self.gen_o, self.gen_x = control_flow_ops.while_loop(
            cond=lambda i, _1, _2, _3, _4: i < self.sequence_length,
            body=_g_recurrence,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       tf.gather(self.g_embeddings, self.start_token),
                       self.h0, gen_o, gen_x)) # begins the recurrent loop

        # discriminator on generated and real data
        d_gen_predictions = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length,
            dynamic_size=False, infer_shape=True)
        d_real_predictions = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length,
            dynamic_size=False, infer_shape=True)

        self.gen_x = self.gen_x.stack()
        emb_gen_x = tf.gather(self.d_embeddings, self.gen_x)
        ta_emb_gen_x = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length)
        ta_emb_gen_x = ta_emb_gen_x.unstack(emb_gen_x)

        emb_real_x = tf.gather(self.d_embeddings, self.x)
        ta_emb_real_x = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length)
        ta_emb_real_x = ta_emb_real_x.unstack(emb_real_x)

        def _d_recurrence(i, inputs, h_tm1, pred):
            x_t = inputs.read(i)
            h_t = self.d_recurrent_unit(x_t, h_tm1)
            y_t = self.d_classifier_unit(h_t)
            pred = pred.write(i, y_t)
            return i + 1, inputs, h_t, pred

        _, _, _, self.d_gen_predictions = control_flow_ops.while_loop(
            cond=lambda i, _1, _2, _3: i < self.sequence_length,
            body=_d_recurrence,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       ta_emb_gen_x,
                       self.d_h0,
                       d_gen_predictions))
        self.d_gen_predictions = tf.reshape(
                self.d_gen_predictions.stack(),
                [self.sequence_length])

        _, _, _, self.d_real_predictions = control_flow_ops.while_loop(
            cond=lambda i, _1, _2, _3: i < self.sequence_length,
            body=_d_recurrence,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       ta_emb_real_x,
                       self.d_h0,
                       d_real_predictions))
        self.d_real_predictions = tf.reshape(
                self.d_real_predictions.stack(),
                [self.sequence_length])

        # supervised pretraining for generator
        g_predictions = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length,
            dynamic_size=False, infer_shape=True)

        emb_x = tf.gather(self.g_embeddings, self.x)
        ta_emb_x = tensor_array_ops.TensorArray(
            dtype=tf.float32, size=self.sequence_length)
        ta_emb_x = ta_emb_x.unstack(emb_x)

        def _pretrain_recurrence(i, x_t, h_tm1, g_predictions):
            h_t = self.g_recurrent_unit(x_t, h_tm1)
            o_t = self.g_output_unit(h_t)
            g_predictions = g_predictions.write(i, o_t)
            x_tp1 = ta_emb_x.read(i)
            return i + 1, x_tp1, h_t, g_predictions

        _, _, _, self.g_predictions = control_flow_ops.while_loop(
            cond=lambda i, _1, _2, _3: i < self.sequence_length,
            body=_pretrain_recurrence,
            loop_vars=(tf.constant(0, dtype=tf.int32),
                       tf.gather(self.g_embeddings, self.start_token),
                       self.h0, g_predictions))

        self.g_predictions = tf.reshape(
                self.g_predictions.stack(),
                [self.sequence_length, self.num_emb])

        # calculate discriminator loss
        self.d_gen_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=self.d_gen_predictions, labels=tf.zeros([self.sequence_length])))
        self.d_real_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=self.d_real_predictions, labels=tf.ones([self.sequence_length])))

        # calculate generator rewards and loss
        decays = tf.exp(tf.log(self.reward_gamma) * tf.to_float(tf.range(self.sequence_length)))
        rewards = _backwards_cumsum(decays * tf.sigmoid(self.d_gen_predictions),
                                    self.sequence_length)
        normalized_rewards = \
            rewards / _backwards_cumsum(decays, self.sequence_length) - self.expected_reward

        self.reward_loss = tf.reduce_mean(normalized_rewards ** 2)
        self.g_loss = \
            -tf.reduce_mean(tf.log(self.gen_o.stack()) * normalized_rewards)

        # pretraining loss
        self.pretrain_loss = \
            (-tf.reduce_sum(
                tf.one_hot(tf.to_int64(self.x),
                           self.num_emb, 1.0, 0.0) * tf.log(self.g_predictions))
             / self.sequence_length)

        # training updates
        d_opt = self.d_optimizer(self.learning_rate)
        g_opt = self.g_optimizer(self.learning_rate)
        pretrain_opt = self.g_optimizer(self.learning_rate)
        reward_opt = tf.train.GradientDescentOptimizer(self.learning_rate)

        self.d_gen_grad = tf.gradients(self.d_gen_loss, self.d_params)
        self.d_real_grad = tf.gradients(self.d_real_loss, self.d_params)
        self.d_gen_updates = d_opt.apply_gradients(zip(self.d_gen_grad, self.d_params))
        self.d_real_updates = d_opt.apply_gradients(zip(self.d_real_grad, self.d_params))

        self.reward_grad = tf.gradients(self.reward_loss, [self.expected_reward])
        self.reward_updates = reward_opt.apply_gradients(zip(self.reward_grad, [self.expected_reward]))

        self.g_grad = tf.gradients(self.g_loss, self.g_params)
        self.g_updates = g_opt.apply_gradients(zip(self.g_grad, self.g_params))

        self.pretrain_grad = tf.gradients(self.pretrain_loss, self.g_params)
        self.pretrain_updates = pretrain_opt.apply_gradients(zip(self.pretrain_grad, self.g_params))

    def generate(self, session):
        outputs = session.run(
                [self.gen_x],
                feed_dict={self.h0: np.random.normal(size=self.hidden_dim),
                           self.samples: np.random.random(self.sequence_length)})
        return outputs[0]

    def train_g_step(self, session):
        outputs = session.run(
                [self.g_updates, self.reward_updates, self.g_loss,
                 self.expected_reward, self.gen_x],
                feed_dict={self.h0: np.random.normal(size=self.hidden_dim),
                           self.samples: np.random.random(self.sequence_length)})
        return outputs

    def train_d_gen_step(self, session):
        outputs = session.run(
                [self.d_gen_updates, self.d_gen_loss],
                feed_dict={self.h0: np.random.normal(size=self.hidden_dim),
                           self.samples: np.random.random(self.sequence_length)})
        return outputs

    def train_d_real_step(self, session, x):
        outputs = session.run([self.d_real_updates, self.d_real_loss],
                              feed_dict={self.x: x})
        return outputs

    def pretrain_step(self, session, x):
        outputs = session.run([self.pretrain_updates, self.pretrain_loss, self.g_predictions],
                              feed_dict={self.x: x,
                                         self.h0: np.random.normal(size=self.hidden_dim)})
        return outputs

    def init_matrix(self, shape):
        return tf.random_normal(shape, stddev=0.1)

    def init_vector(self, shape):
        return tf.zeros(shape)

    def create_recurrent_unit(self, params):
        self.W_rec = tf.Variable(self.init_matrix([self.hidden_dim, self.emb_dim]))
        params.append(self.W_rec)
        def unit(x_t, h_tm1):
            return h_tm1 + tf.reshape(tf.matmul(self.W_rec, tf.reshape(x_t, [self.emb_dim, 1])), [self.hidden_dim])
        return unit

    def create_output_unit(self, params, embeddings):
        self.W_out = tf.Variable(self.init_matrix([self.emb_dim, self.hidden_dim]))
        self.b_out1 = tf.Variable(self.init_vector([self.emb_dim, 1]))
        self.b_out2 = tf.Variable(self.init_vector([self.num_emb, 1]))
        params.extend([self.W_out, self.b_out1, self.b_out2])
        def unit(h_t):
            logits = tf.reshape(
                    self.b_out2 +
                    tf.matmul(embeddings,
                              tf.tanh(self.b_out1 +
                                      tf.matmul(self.W_out, tf.reshape(h_t, [self.hidden_dim, 1])))),
                    [1, self.num_emb])
            return tf.reshape(tf.nn.softmax(logits), [self.num_emb])
        return unit

    def create_classifier_unit(self, params):
        self.W_class = tf.Variable(self.init_matrix([1, self.hidden_dim]))
        self.b_class = tf.Variable(self.init_vector([1]))
        params.extend([self.W_class, self.b_class])
        def unit(h_t):
            return self.b_class + tf.matmul(self.W_class, tf.reshape(h_t, [self.hidden_dim, 1]))
        return unit

    def d_optimizer(self, *args, **kwargs):
        return tf.train.GradientDescentOptimizer(*args, **kwargs)

    def g_optimizer(self, *args, **kwargs):
        return tf.train.GradientDescentOptimizer(*args, **kwargs)


class GRU(RNN):

    def create_recurrent_unit(self, params):
        self.W_rx = tf.Variable(self.init_matrix([self.hidden_dim, self.emb_dim]))
        self.W_zx = tf.Variable(self.init_matrix([self.hidden_dim, self.emb_dim]))
        self.W_hx = tf.Variable(self.init_matrix([self.hidden_dim, self.emb_dim]))
        self.U_rh = tf.Variable(self.init_matrix([self.hidden_dim, self.hidden_dim]))
        self.U_zh = tf.Variable(self.init_matrix([self.hidden_dim, self.hidden_dim]))
        self.U_hh = tf.Variable(self.init_matrix([self.hidden_dim, self.hidden_dim]))
        params.extend([
            self.W_rx, self.W_zx, self.W_hx,
            self.U_rh, self.U_zh, self.U_hh])

        def unit(x_t, h_tm1):
            x_t = tf.reshape(x_t, [self.emb_dim, 1])
            h_tm1 = tf.reshape(h_tm1, [self.hidden_dim, 1])
            r = tf.sigmoid(tf.matmul(self.W_rx, x_t) + tf.matmul(self.U_rh, h_tm1))
            z = tf.sigmoid(tf.matmul(self.W_zx, x_t) + tf.matmul(self.U_zh, h_tm1))
            h_tilda = tf.tanh(tf.matmul(self.W_hx, x_t) + tf.matmul(self.U_hh, r * h_tm1))
            h_t = (1 - z) * h_tm1 + z * h_tilda
            return tf.reshape(h_t, [self.hidden_dim])

        return unit


This function is quite a bit easier to understand. It trains the model over a specified number of generator and discriminator steps, in this case 3 discriminator steps for every 1 generator. Half of the discriminator steps are on real data and half are on generated. Every epoch, the proportion of iterations that the generator is trained in a supervised fashion decreases, and lets the discriminator lead the generator totally when this proportion reaches 0.

The supervised generated samples are extracted via taking the argmax of the rows of the generators output layer, ie. a SEQ_LENGTH no. of neurons that have the highest probability of being chosen. These correspond to 20 generated characters. 
The unsupervised generated samples is simply the output layer's after it has been led by the discriminator. The output layer is sampled via the \_cumsum method and then returned at the end of every epoch. (for greater insight, look at the \_g_recurrence method above )

In [0]:
import time
def train_epoch(sess, trainable_model, num_iter,
                proportion_supervised, g_steps, d_steps,
                next_sequence, verify_sequence=None,
                words=None,
                proportion_generated=0.5):
    """Perform training for model.

    sess: tensorflow session
    trainable_model: the model
    num_iter: number of iterations
    proportion_supervised: what proportion of iterations should the generator
        be trained in a supervised manner (rather than trained via discriminator)
    g_steps: number of generator training steps per iteration
    d_steps: number of discriminator training steps per iteration
    next_sequence: function that returns a groundtruth sequence
    verify_sequence: function that checks a generated sequence, returning True/False
    words:  array of words (to map indices back to words)
    proportion_generated: what proportion of steps for the discriminator
        should be on artificially generated data

    """
    start = time.time()
    supervised_g_losses = [0]  # we put in 0 to avoid empty slices
    unsupervised_g_losses = [0]  # we put in 0 to avoid empty slices
    d_losses = [0]
    expected_rewards = [[0] * trainable_model.sequence_length]
    supervised_correct_generation = [0]
    unsupervised_correct_generation = [0]
    supervised_gen_x = None
    unsupervised_gen_x = None
    print('running %d iterations with %d g steps and %d d steps' % (num_iter, g_steps, d_steps))
    print('of the g steps, %.2f will be supervised' % proportion_supervised)
    for it in range(num_iter):
        for _ in range(g_steps):
            if random.random() < proportion_supervised:
              
                seq = next_sequence()         
                _, g_loss, g_pred = trainable_model.pretrain_step(sess, seq)
                supervised_g_losses.append(g_loss)

                supervised_gen_x = np.argmax(g_pred, axis=1)
                if verify_sequence is not None:
                    supervised_correct_generation.append(
                        verify_sequence(supervised_gen_x))
            else:
                _, _, g_loss, expected_reward, unsupervised_gen_x = \
                    trainable_model.train_g_step(sess)
                expected_rewards.append(expected_reward)
                unsupervised_g_losses.append(g_loss)

                if verify_sequence is not None:
                    unsupervised_correct_generation.append(
                        verify_sequence(unsupervised_gen_x))

        for _ in range(d_steps):
            if random.random() < proportion_generated:
                seq = next_sequence()
                _, d_loss = trainable_model.train_d_real_step(sess, seq)
            else:
                _, d_loss = trainable_model.train_d_gen_step(sess)
            d_losses.append(d_loss)

    print('epoch statistics:')
    end = time.time()
    print('Elapsed time:', end-start)
    print('>>>> discriminator loss:', np.mean(d_losses))
    print('>>>> generator loss:', np.mean(supervised_g_losses), np.mean(unsupervised_g_losses))
    if verify_sequence is not None:
        print('>>>> correct generations (supervised, unsupervised):', np.mean(supervised_correct_generation), np.mean(unsupervised_correct_generation))
    print('>>>> sampled generations (supervised, unsupervised):',)
    print(''.join([words[x] if words else x for x in supervised_gen_x]) if supervised_gen_x is not None else None,)
    print(''.join([words[x] if words else x for x in unsupervised_gen_x]) if unsupervised_gen_x is not None else None)
    print('>>>> expected rewards:', np.mean(expected_rewards, axis=0))


Here we actually run our session and train our model. We start by importing a couple of libraries and starting up a Tensorboard tunnel. Then, we define our constants. We download our dataset and process it into a token (character) stream, and remove any non-english (printable) characters. 

For EPOCH_ITER iterations, we train our classifier, using a random sequence every time, the constants defined, and our word index dictionary. The training function above prints out how we're doing each time.

In [0]:
!pip install -U tensorboardcolab
import codecs
from tensorboardcolab import *
import shutil

#clean out the directory
shutil.rmtree('./Graph', ignore_errors=True)
os.mkdir('./Graph')

import os.path
import numpy as np
import tensorflow as tf
import random
import subprocess
import gzip
import string

tbc=TensorBoardColab(startup_waiting_time=10)

EMB_DIM = 25 # SIZE OF THE EMBEDDING LAYER IN THE NETWORK (INPUT)
HIDDEN_DIM = 25 # NUMBER OF HIDDEN NEURONS
SEQ_LENGTH = 30 # HOW LONG WE WANT OUR GENERATED SEQUENCES TO BE
START_TOKEN = 0 # INDEX OF _START

EPOCH_ITER = 1000 # NUMBER OF EPOCHS WE WISH TO RUN
CURRICULUM_RATE = 0.01  # how quickly to move from supervised training to unsupervised
TRAIN_ITER = 1000000  # generator/discriminator alternating 
D_STEPS = 3  # how many times to train the discriminator per generator step
SEED = 42

DATA_FILE = 'normalizedZIPReviews.txt'
printable = set(string.printable)

def tokenize(s):
    return [c for c in ' '.join(s.split())]
  

def get_data(download=not os.path.exists(DATA_FILE)):
    """Downloads and parses Moby Dick."""
    if download:
        subprocess.check_output(
            ['wget', 'https://storage.googleapis.com/lucas0/normalizedZIPReviews.txt',
             '-O', DATA_FILE])

    token_stream = []
    is_gzip = False
    try:
        open(DATA_FILE).read(2)
    except UnicodeDecodeError:
        is_gzip = True
    with gzip.open(DATA_FILE) if is_gzip else codecs.open(DATA_FILE, 'r', 'utf-8',errors='ignore') as f:
        for line in f:
            line = line if not is_gzip else line.decode('utf-8')
            if line.strip():
                line = ''.join([i if ord(i) < 128 else ' ' for i in line])
                token_stream.extend(tokenize(line.strip().lower()))
                token_stream.append(' ')

    return token_stream


class ReviewGRU(GRU):

    def d_optimizer(self, *args, **kwargs):
        return tf.train.AdamOptimizer()  # ignore learning rate

    def g_optimizer(self, *args, **kwargs):
        return tf.train.AdamOptimizer()  # ignore learning rate


def get_trainable_model(num_emb):
    return ReviewGRU(
        num_emb, EMB_DIM, HIDDEN_DIM,
        SEQ_LENGTH, START_TOKEN)


def get_random_sequence(token_stream, word2idx):
    """Returns random subsequence."""
    start_idx = random.randint(0, len(token_stream) - SEQ_LENGTH)
    return [word2idx[tok] for tok in token_stream[start_idx:start_idx + SEQ_LENGTH]]


def verify_sequence(three_grams, seq):
    """Not a true verification; only checks 3-grams."""
    for i in range(len(seq) - 3):
        if tuple(seq[i:i + 3]) not in three_grams:
            return False
    return True


def main():
    random.seed(SEED)
    np.random.seed(SEED)

    token_stream = get_data()
    assert START_TOKEN == 0
    words = ['_START'] + list(set(token_stream))
    print(words)
    word2idx = dict((word, i) for i, word in enumerate(words))
    num_words = len(words)
    three_grams = dict((tuple(word2idx[w] for w in token_stream[i:i + 3]), True)
                       for i in range(len(token_stream) - 3))
    print('num words', num_words)
    print('stream length', len(token_stream))
    print('distinct 3-grams', len(three_grams))

    trainable_model = get_trainable_model(num_words)
    sess = tf.Session()
    mdProto = tf.RunMetadata()
    sess.run(tf.global_variables_initializer())
    
    train_writer = tbc.get_writer();
    train_writer.add_graph(sess.graph)
    
    print('training')
    for epoch in range(TRAIN_ITER // EPOCH_ITER):
        print('epoch', epoch)
        proportion_supervised = max(0.0, 1.0 - CURRICULUM_RATE * epoch)
        train_epoch(
            sess, trainable_model, EPOCH_ITER,
            proportion_supervised=proportion_supervised,
            g_steps=1, d_steps=D_STEPS,
            next_sequence=lambda: get_random_sequence(token_stream, word2idx),
            verify_sequence=lambda seq: verify_sequence(three_grams, seq),
            words=words)


Requirement already up-to-date: tensorboardcolab in /usr/local/lib/python3.6/dist-packages (0.0.22)
Wait for 10 seconds...
TensorBoard link:
https://6869b4de.ngrok.io


In [0]:
main()

['_START', 'b', '"', '(', '$', '0', 'a', '.', 'f', 'd', '_', 'y', ')', ' ', '8', '=', 'c', '9', 'w', '^', 'j', '&', 'p', '4', '\\', '+', '5', '7', 'k', '-', '1', ']', '`', ';', 'n', '3', '[', "'", '*', '2', 'g', ':', '@', '#', 'z', '~', '?', '%', '!', '6', 'm', 'q', '/', '{', 'r', 'i', 'e', 'u', 'x', 'v', 'o', ',', '}', 't', '|', 'h', 's', 'l']
num words 68
stream length 11313890
distinct 3-grams 22654
training
epoch 0
running 1000 iterations with 1 g steps and 3 d steps
of the g steps, 1.00 will be supervised
epoch statistics:
Elapsed time: 198.29249739646912
>>>> discriminator loss: 0.5159630743027608
>>>> generator loss: 3.1926892353938174 0.0
>>>> correct generations (supervised, unsupervised): 0.016983016983016984 0.0
>>>> sampled generations (supervised, unsupervised):
ee                            
None
>>>> expected rewards: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
epoch 1
running 1000 iterations with 1 g steps and 3 d steps
o

Because Colab kicks you off their GPU's after 12 hours, I did not get to fully train this model. However, after this run I did some research and discovered some issues to do with this form of GAN. Hyperparameters are extremely important in this form of GAN due to mode collapse, and the wrong parameter will entirely diminish the effectiveness of the model. I'm not overly sure how to address this without running these massive models repeatedly with different parameters, which will take a huge amount of time. I will bring this up in our meeting with Hojjat this evening.

